<a href="https://colab.research.google.com/github/Alihassan7726/Text-classification-using-ULMFiT/blob/main/Text_Classification_using_ULMFiT_on_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os
pd.set_option('display.width',1000)
pd.set_option('max_colwidth', 1000) 
import pandas as pd

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
language_model_learner??

In [ ]:
path = untar_data(URLs.IMDB)
path

PosixPath('/root/.fastai/data/imdb')

In [ ]:
# bs = 48
bs = 128

In [ ]:
data_lm = (TextList.from_folder(path)
           #Inputs: all the text files in path
            .filter_by_folder(include=['train', 'test', 'unsup']) 
           #We may have other temp folders that contain text files so we only keep what's in train, test and unsup
            .split_by_rand_pct(0.1 , seed = 42)
           #We randomly split and keep 10% (10,000 reviews) for validation
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=bs, bptt=bptt))

data_lm.save('data_lm.pkl')
len(data_lm.vocab.itos) , len(data_lm.train_ds) #, len(data_lm.tain_ds.vocab)


/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


AttributeError: ignored

In [ ]:
data_lm.save('lm_databunch')

In [ ]:
data_lm = load_data(path , 'lm_databunch' , bs = bs)

In [ ]:
# drop_mult is a reguralization parameter DROPOUT
learn = language_model_learner(data_lm,  arch = AWD_LSTM, drop_mult=1.).to_fp16()  # fp16() for speed a mixed precision training let's say

In [ ]:
lr = 1e-2
lr *= bs/48
lr

0.026666666666666665

In [ ]:
learn.fit_one_cycle(1, lr , moms = (0.8 , 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.444500,4.073424,0.287040,14:24


In [ ]:
#learn_lm.unfreeze()
# learn.fit_one_cycle(10, lr , moms = (0.8 , 0.7)) Fine-tunning for 10 epochs without overfitting the LM

In [ ]:
learn.save('fine_tunned_10') # Saving model parameters and weights etc

learn.save_encoder('fine_tunned_enc_10') # Saving the encoder model that will interprets the sequence and gives a final vector representation

In [ ]:
data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['neg', 'pos'])
             #label them all with their folders
             .databunch(bs=bs))


/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
data_clas.save('imdb_TextClass')
data_clas = load_data(path , 'imdb_TextClass' , bs = bs)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
# Let’s now use the data_clas object we created earlier to build a classifier with our fine-tuned encoder.
learn_c = text_classifier_learner(data_clas, drop_mult=0.7 , arch = AWD_LSTM)
learn_c.load_encoder('fine_tunned_enc_10')
learn_c.freeze()

In [ ]:
lr = 2e-2
lr *= bs/48
lr

0.05333333333333333

In [ ]:
learn_c.fit_one_cycle(1, lr , moms = (0.8 , 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.478121,0.356933,0.845200,04:26


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(1, lr , moms = (0.8 , 0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.352331,0.234056,0.904880,04:51


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
import gc
variables = gc.collect()
del variables


In [ ]:
import torch
torch.cuda.empty_cache() # To avoid CUDA GPU memory errors

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 4            |        cudaMalloc retries: 4         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   13679 MB |   13837 MB |   41249 GB |   41236 GB |\n|       from large pool |   13606 MB |   13763 MB |   41205 GB |   41191 GB |\n|       from small pool |      73 MB |      74 MB |      44 GB |      44 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |   13679 MB |   13837 MB |   41249 GB |   41236 GB |\n|       from large pool |   13606 MB |   13763 MB |

In [ ]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(1, lr , moms = (0.8 , 0.7))

epoch,train_loss,valid_loss,accuracy,time


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


RuntimeError: ignored